<h2> Respiratory disease classification <h2>

Imports

In [50]:
import pandas as pd
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, pipeline
import re

Load the data

In [4]:
file = "C:\\Users\\Usuario\\universidad\\MUIT\\2º\\MLLB\\OSA CASE STUDY\\R\\DATA\\df_final.csv"
df_final = pd.read_csv(file)

In [5]:
df_final

,Patient,Gender,IAH,Weight,Height,Age,Cervical,Smoker,Snorer,Illnesses
0,P0002,male,29.6,119,174,56,48.0,yes,ns,nd
1,P0004,male,19.7,78,168,39,42.0,no,ns,Hiperactividad_bronquial
2,P0005,male,9.0,80,173,32,40.0,no,ns,Septo_Nasal_Desviado_Operado
3,P0006,male,2.0,109,190,32,42.0,no,ns,Hipertrofia_cornetes_nasales
4,P0007,male,34.0,86,169,39,42.0,no,ns,Cornetes
...,...,...,...,...,...,...,...,...,...,...
618,P0677,female,36.3,82,165,64,39.0,former,yes,Atrofia trompas nariz y faringitis en prueba
619,P0679,male,9.2,105,180,35,45.0,no,yes,Apnea
620,P0680,male,52.2,90,180,50,42.0,no,yes,no
621,P0682,female,5.4,66,164,57,35.0,yes,yes,Lupus


Initialize GPT-2 Model

In [12]:
model_name = "gpt2"
model = GPT2ForSequenceClassification.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


1

In [14]:
def classification_function(text, model, tokenizer):
    labels = ["Respiratory", "Not Respiratory"]
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    logits = model(**inputs).logits
    predicted_class = logits.argmax().item()
    return labels[predicted_class]

In [17]:
df_final['Illnesses'] = df_final['Illnesses'].str.lower()  # Convierte a minúsculas
df_final['Illnesses'] = df_final['Illnesses'].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)  # Elimina caracteres raros
#df_final['Illnesses'] = df_final['Illnesses'].str.replace(r'\s+', ' ', regex=True)  # Elimina espacios en blanco extra

df_final['Clasification'] = df_final['Illnesses'].apply(lambda x: classification_function(x, model, tokenizer))


In [18]:
df_final

,Patient,Gender,IAH,Weight,Height,Age,Cervical,Smoker,Snorer,Illnesses,Clasification
0,P0002,male,29.6,119,174,56,48.0,yes,ns,nd,Respiratory
1,P0004,male,19.7,78,168,39,42.0,no,ns,hiperactividadbronquial,Not Respiratory
2,P0005,male,9.0,80,173,32,40.0,no,ns,septonasaldesviadooperado,Not Respiratory
3,P0006,male,2.0,109,190,32,42.0,no,ns,hipertrofiacornetesnasales,Not Respiratory
4,P0007,male,34.0,86,169,39,42.0,no,ns,cornetes,Not Respiratory
...,...,...,...,...,...,...,...,...,...,...,...
618,P0677,female,36.3,82,165,64,39.0,former,yes,atrofia trompas nariz y faringitis en prueba,Not Respiratory
619,P0679,male,9.2,105,180,35,45.0,no,yes,apnea,Not Respiratory
620,P0680,male,52.2,90,180,50,42.0,no,yes,no,Respiratory
621,P0682,female,5.4,66,164,57,35.0,yes,yes,lupus,Not Respiratory


In [51]:
from transformers import BertForSequenceClassification, BertTokenizer
import torch

In [52]:
file = "C:\\Users\\Usuario\\universidad\\MUIT\\2º\\MLLB\\OSA CASE STUDY\\R\\DATA\\df_final.csv"
df_final = pd.read_csv(file)


In [26]:
#model_name = "bert-base-uncased" #Este está preentrenado en inglés
model_name = "dccuchile/bert-base-spanish-wwm-uncased" #Este está preentrenado en español
model = BertForSequenceClassification.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

c:\Users\Usuario\AppData\Local\Programs\Python\Python39\lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Usuario\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', 'bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [53]:
def classification_function(text, model, tokenizer):
    labels = ["Si", "No"]

    inputs = tokenizer(text, return_tensors="pt", max_length=200, truncation=True, padding="max_length")
    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]

    with torch.no_grad():
        logits = model(input_ids=input_ids, attention_mask=attention_mask).logits
        predicted_class = torch.argmax(logits, dim=1).item()

    return labels[predicted_class]

In [54]:
df_final['Illnesses'] = df_final['Illnesses'].str.replace('_', ' ', regex=False)
df_final["Illnesses"] = df_final["Illnesses"].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)  # Elimina caracteres raros
df_final["Illnesses"] = df_final["Illnesses"].str.replace(r'\s+', ' ', regex=True)  # Elimina espacios en blanco extra

df_final["Classification"] = df_final["Illnesses"].apply(lambda x: classification_function(x, model, tokenizer))

In [56]:
df_final

,Patient,Gender,IAH,Weight,Height,Age,Cervical,Smoker,Snorer,Illnesses,Classification
0,P0002,male,29.6,119,174,56,48.0,yes,ns,nd,No
1,P0004,male,19.7,78,168,39,42.0,no,ns,Hiperactividad bronquial,No
2,P0005,male,9.0,80,173,32,40.0,no,ns,Septo Nasal Desviado Operado,No
3,P0006,male,2.0,109,190,32,42.0,no,ns,Hipertrofia cornetes nasales,No
4,P0007,male,34.0,86,169,39,42.0,no,ns,Cornetes,No
...,...,...,...,...,...,...,...,...,...,...,...
618,P0677,female,36.3,82,165,64,39.0,former,yes,Atrofia trompas nariz y faringitis en prueba,No
619,P0679,male,9.2,105,180,35,45.0,no,yes,Apnea,No
620,P0680,male,52.2,90,180,50,42.0,no,yes,no,No
621,P0682,female,5.4,66,164,57,35.0,yes,yes,Lupus,No


CLASSIFICATION WITH ZERO-SHOT

GPT2

In [99]:
import torch
import pandas as pd
from transformers import GPT2Tokenizer, pipeline

In [100]:
file = "C:\\Users\\Usuario\\universidad\\MUIT\\2º\\MLLB\\OSA CASE STUDY\\R\\DATA\\df_final.csv"
df_final = pd.read_csv(file)

In [101]:
df_final.drop(['Snorer'], axis = 1)

,Patient,Gender,IAH,Weight,Height,Age,Cervical,Smoker,Illnesses
0,P0002,male,29.6,119,174,56,48.0,yes,nd
1,P0004,male,19.7,78,168,39,42.0,no,Hiperactividad_bronquial
2,P0005,male,9.0,80,173,32,40.0,no,Septo_Nasal_Desviado_Operado
3,P0006,male,2.0,109,190,32,42.0,no,Hipertrofia_cornetes_nasales
4,P0007,male,34.0,86,169,39,42.0,no,Cornetes
...,...,...,...,...,...,...,...,...,...
618,P0677,female,36.3,82,165,64,39.0,former,Atrofia trompas nariz y faringitis en prueba
619,P0679,male,9.2,105,180,35,45.0,no,Apnea
620,P0680,male,52.2,90,180,50,42.0,no,no
621,P0682,female,5.4,66,164,57,35.0,yes,Lupus


In [102]:
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = "<|endoftext|>"
classifier = pipeline("zero-shot-classification", model=model_name, tokenizer=tokenizer)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.


In [103]:
def classification_function(illness, classifier):
    result = classifier(illness, ["respiratory","not respiratory"])
    label_classified = result["labels"][0]
    return label_classified

In [104]:
df_final['Illnesses'] = df_final['Illnesses'].str.replace('_', ' ', regex=False)
df_final["Illnesses"] = df_final["Illnesses"].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)  # Elimina caracteres raros
df_final["Illnesses"] = df_final["Illnesses"].str.replace(r'\s+', ' ', regex=True)  # Elimina espacios en blanco extra

In [105]:
df_final["Classification"] = df_final["Illnesses"].apply(lambda x: classification_function(x, classifier))

In [106]:
df_final[1:20]

,Patient,Gender,IAH,Weight,Height,Age,Cervical,Smoker,Snorer,Illnesses,Classification
1,P0004,male,19.7,78,168,39,42.0,no,ns,Hiperactividad bronquial,respiratory
2,P0005,male,9.0,80,173,32,40.0,no,ns,Septo Nasal Desviado Operado,respiratory
3,P0006,male,2.0,109,190,32,42.0,no,ns,Hipertrofia cornetes nasales,respiratory
4,P0007,male,34.0,86,169,39,42.0,no,ns,Cornetes,respiratory
5,P0008,male,60.0,145,172,47,44.0,no,ns,Rinitis,respiratory
6,P0009,male,22.0,72,165,40,42.0,no,ns,Rinitis crnica,respiratory
7,P0010,male,11.0,80,180,28,38.0,no,ns,no,respiratory
8,P0014,female,7.0,50,158,50,35.0,yes,ns,Sinusitis,respiratory
9,P0015,female,5.0,55,156,62,38.0,yes,ns,no,respiratory
10,P0016,male,26.0,95,166,77,46.0,no,ns,Asma,respiratory


BERT. SPANISH

In [107]:
import pandas as pd
from transformers import pipeline, BertTokenizer

In [108]:
file = "C:\\Users\\Usuario\\universidad\\MUIT\\2º\\MLLB\\OSA CASE STUDY\\R\\DATA\\df_final.csv"
df_final = pd.read_csv(file)

In [109]:
df_final.drop(['Snorer'], axis = 1)

,Patient,Gender,IAH,Weight,Height,Age,Cervical,Smoker,Illnesses
0,P0002,male,29.6,119,174,56,48.0,yes,nd
1,P0004,male,19.7,78,168,39,42.0,no,Hiperactividad_bronquial
2,P0005,male,9.0,80,173,32,40.0,no,Septo_Nasal_Desviado_Operado
3,P0006,male,2.0,109,190,32,42.0,no,Hipertrofia_cornetes_nasales
4,P0007,male,34.0,86,169,39,42.0,no,Cornetes
...,...,...,...,...,...,...,...,...,...
618,P0677,female,36.3,82,165,64,39.0,former,Atrofia trompas nariz y faringitis en prueba
619,P0679,male,9.2,105,180,35,45.0,no,Apnea
620,P0680,male,52.2,90,180,50,42.0,no,no
621,P0682,female,5.4,66,164,57,35.0,yes,Lupus


In [110]:
model_name = "dccuchile/bert-base-spanish-wwm-uncased"
classifier = pipeline("zero-shot-classification", model=model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased and are newly initialized: ['classifier.weight', 'bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.


In [111]:
def classification_function(text, classifier, tokenizer):
    labels = ["no respiratoria", "respiratoria"]

    # Limpiar y tokenizar el texto
    text = text.lower()
    text = text.replace("_", " ")  # Reemplazar guiones bajos por espacios
    inputs = tokenizer(text, return_tensors="pt", max_length=200, truncation=True, padding=True)

    # Realizar la clasificación zero-shot
    result = classifier(text, labels)

    # Obtener la etiqueta clasificada
    predicted_label = result["labels"][0]

    return predicted_label

In [112]:
df_final['Illnesses'] = df_final['Illnesses'].str.replace('_', ' ', regex=False)
df_final["Illnesses"] = df_final["Illnesses"].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)  # Elimina caracteres raros
df_final["Illnesses"] = df_final["Illnesses"].str.replace(r'\s+', ' ', regex=True)  # Elimina espacios en blanco extra

In [113]:
df_final["Respiratory"] = df_final["Illnesses"].apply(lambda x: classification_function(x, classifier, tokenizer))

In [114]:
df_final[1:20]

,Patient,Gender,IAH,Weight,Height,Age,Cervical,Smoker,Snorer,Illnesses,Classification
1,P0004,male,19.7,78,168,39,42.0,no,ns,Hiperactividad bronquial,no respiratoria
2,P0005,male,9.0,80,173,32,40.0,no,ns,Septo Nasal Desviado Operado,no respiratoria
3,P0006,male,2.0,109,190,32,42.0,no,ns,Hipertrofia cornetes nasales,no respiratoria
4,P0007,male,34.0,86,169,39,42.0,no,ns,Cornetes,no respiratoria
5,P0008,male,60.0,145,172,47,44.0,no,ns,Rinitis,no respiratoria
6,P0009,male,22.0,72,165,40,42.0,no,ns,Rinitis crnica,no respiratoria
7,P0010,male,11.0,80,180,28,38.0,no,ns,no,respiratoria
8,P0014,female,7.0,50,158,50,35.0,yes,ns,Sinusitis,no respiratoria
9,P0015,female,5.0,55,156,62,38.0,yes,ns,no,respiratoria
10,P0016,male,26.0,95,166,77,46.0,no,ns,Asma,no respiratoria


In [ ]:
file = "C:\\Users\\Usuario\\universidad\\MUIT\\2º\\MLLB\\OSA CASE STUDY\\R\\DATA\\df_final.xlsx"
df_final.to_excel(file, index=False)